# Game Logs
This notebook explores the use of these two endpoints in retrieving detailed game stats for a given player:
- https://v2.nba.api-sports.io/games
- https://v2.nba.api-sports.io/players/statistics

The code from this notebook will be used in V1 of the _Player Search_ feature.

In [1]:
from rel_bball_analytics.api import get_data_from_api
from rel_bball_analytics.statistics.data_source import get_player_stats

from datetime import datetime
import pandas as pd

In [2]:
PLAYER_ID = 124
TEAM_ID = 11
SEASON = 2022

### Get Game Info from Team and Season

In [3]:
GAME_COLUMNS = [
    "id",
    "season",
    "date",
    "home",
    "away",
    "home_score",
    "away_score",
    "winner",
    "difference",
]

In [4]:
def parse_date(date):
    return datetime.strptime(date, "%Y-%m-%dT%H:%M:%S.000Z")

In [5]:
def get_winner(row):
    return row["home"] if row["home_score"] > row["away_score"] else row["away"]

In [6]:
def filter_finished_games(games):
    games["is_finished"] = games["status"].apply(lambda obj: obj["long"] == "Finished")
    return games.loc[games.is_finished]

In [7]:
def clean_game_data(data):
    data["date"] = data["date"].apply(lambda obj: parse_date(obj.get("start")))
    data["home"] = data["teams"].apply(lambda obj: obj["home"].get("code"))
    data["away"] = data["teams"].apply(lambda obj: obj["visitors"].get("code"))
    data["home_score"] = data["scores"].apply(lambda obj: obj["home"].get("points"))
    data["away_score"] = data["scores"].apply(lambda obj: obj["visitors"].get("points"))
    data["difference"] = abs(data["home_score"] - data["away_score"])
    data["winner"] = data.apply(get_winner, axis=1)

    return data[GAME_COLUMNS]

In [8]:
params = {"team": TEAM_ID, "season": SEASON, "league": "Standard"}
game_data = get_data_from_api(endpoint="games", params=params)

if game_data["results"] == 0:
    print("No results found")

df_games = pd.DataFrame(game_data["response"])
df_games = filter_finished_games(games=df_games)
df_games = clean_game_data(data=df_games)

In [9]:
df_games.head()

,id,season,date,home,away,home_score,away_score,winner,difference
0,10980,2022,2022-09-30 10:00:00,WAS,GSW,87,96,GSW,9
1,10983,2022,2022-10-02 05:00:00,GSW,WAS,104,95,GSW,9
2,11019,2022,2022-10-10 00:30:00,GSW,LAL,121,124,LAL,3
3,11030,2022,2022-10-12 02:00:00,GSW,POR,131,98,GSW,33
4,11048,2022,2022-10-15 02:00:00,GSW,DEN,112,119,DEN,7


### Merge Game Data with Player Stats

In [10]:
df_stats = get_player_stats(player_id=PLAYER_ID, season=SEASON)
df_game_log = df_stats.merge(df_games, left_on="game_id", right_on="id")

### Style Game Log

In [11]:
GAME_LOG_COLUMNS = [
    "date", 
    "team",
    "opponent", 
    "result",
    "position",
    "points",
    "minutes_played",
    "field_goals_made",
    "field_goal_attempts",
    "field_goal_percentage",
    "three_points_made",
    "three_point_attempts",
    "three_point_percentage",
    "free_throws_made",
    "free_throw_attempts",
    "free_throw_percentage",
    "offensive_rebounds",
    "defensive_rebounds",
    "total_rebounds",
    "assists",
    "steals",
    "blocks",
    "turnovers",
    "personal_fouls",
]

In [12]:
def format_date(date):
    return date.date().strftime("%Y-%m-%d") 

In [13]:
def format_result(row):
    result = "W" if row["team"] == row["winner"] else "L"
    sign = "+" if row["team"] == row["winner"] else "-"

    return f"{result} ({sign}{row['difference']})"

In [14]:
def get_opponent(row):
    return row["home"] if row["team"] == row["away"] else row["away"]

In [15]:
df_game_log["date"] = df_game_log["date"].apply(format_date)
df_game_log["result"] = df_game_log.apply(format_result, axis=1)
df_game_log["opponent"] = df_game_log.apply(get_opponent, axis=1)
df_game_log = df_game_log[GAME_LOG_COLUMNS]

In [16]:
df_game_log.head()

,date,team,opponent,result,position,points,minutes_played,field_goals_made,field_goal_attempts,field_goal_percentage,...,free_throw_attempts,free_throw_percentage,offensive_rebounds,defensive_rebounds,total_rebounds,assists,steals,blocks,turnovers,personal_fouls
0,2022-10-02,GSW,WAS,W (+9),PG,17,16.97,7,13,53.8,...,0,0.0,0,5,5,2,0,1,4,1
1,2022-09-30,GSW,WAS,W (+9),PG,6,13.08,1,7,14.3,...,3,100.0,0,3,3,2,0,0,4,0
2,2023-01-31,GSW,OKC,W (+8),PG,38,37.00,12,20,60.0,...,6,100.0,1,7,8,12,0,1,4,0
3,2023-01-28,GSW,TOR,W (+12),PG,35,39.00,13,21,61.9,...,5,100.0,0,7,7,11,2,0,3,1
4,2023-01-26,GSW,MEM,W (+2),PG,34,35.00,10,19,52.6,...,11,90.9,0,2,2,3,2,0,3,3
